In [1]:
import json

In [3]:
with open("documents.json", "rt") as f_in:
    docs_raw = json.load(f_in)

In [4]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [5]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [7]:
from elasticsearch import Elasticsearch

In [8]:
es_client = Elasticsearch('http://localhost:9200')

In [9]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'course-questions'}

In [10]:
from tqdm.auto import tqdm
for doc in tqdm(documents):
    es_client.index(index=index_name, body=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [11]:
def search_es(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [12]:
from openai import OpenAI

In [13]:
client = OpenAI(
    base_url="http://localhost:11434/v1/",
    api_key="ollama"
)

In [15]:
def build_prompt(query, results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
    Use only the facts from the CONTEXT when answering the QUESTION. 
    If the CONTEXT doesn't contain the ansewer, output NONE.

    QUESTION: {question}

    CONTEXT: {context}
    """

    context = ""

    for doc in results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [16]:
query = "the course already started, can I still join?"

In [17]:
def llm(prompt):
    response = client.chat.completions.create(
        model='phi3', 
        messages=[{'role': "user", 'content': prompt}]
    )

    return response.choices[0].message.content

In [18]:
def rag(query):
    search_results = search_es(query)
    prompt = build_prompt(query, search_results)
    return llm(prompt)

In [19]:
rag(query)

" Based on the context provided, you are still eligible to join the course even after the starting date as long as you haven't yet submitted your final projects. Ensure that you submit everything in time and avoid leaving it for the last minute. However, note that there might not be additional support or specific mechanisms mentioned in this FAQ database if someone takes the self-paced course beyond its official end date within a semester since we don't have explicit information about post-course activities provided here. The only point made is regarding materials being kept after the course, allowing you to follow it at your own pace even after completion; this doesn't necessarily imply continued enrollment or additional support mechanisms beyond standard practices for self-paced courses like slack and FAQs (as mentioned in context 4).\n\nIf seeking official guidance on joining a new cohort later, the course instructor(s) may provide updated information through different channels such

In [20]:
print(_)

 Based on the context provided, you are still eligible to join the course even after the starting date as long as you haven't yet submitted your final projects. Ensure that you submit everything in time and avoid leaving it for the last minute. However, note that there might not be additional support or specific mechanisms mentioned in this FAQ database if someone takes the self-paced course beyond its official end date within a semester since we don't have explicit information about post-course activities provided here. The only point made is regarding materials being kept after the course, allowing you to follow it at your own pace even after completion; this doesn't necessarily imply continued enrollment or additional support mechanisms beyond standard practices for self-paced courses like slack and FAQs (as mentioned in context 4).

If seeking official guidance on joining a new cohort later, the course instructor(s) may provide updated information through different channels such as